In [71]:
from pathlib import Path 
import requests
#import pandas as pd
import numpy as np
from io import StringIO
from simple_salesforce import Salesforce
import shutil
import datetime 
from datetime import date,datetime
import pandas as pd



In [72]:

sf = Salesforce(
    username="jmartinez@careassistance.com",
    password="Jcmm010483*",
    security_token="1B6xNEvzIe5uroF6nRgKGTSk",
)

In [73]:
results = sf.query_all(
    """
    select id,Paciente__r.IdentificationId__pc, padre__c,Paciente__c,Paciente__r.Habilitado__c,Programas_por_Empresas__r.Datos_Empresa__c,datos2__c,  Programas_por_Empresas__c,Programas_por_Empresas__r.Name,  Empleador__c  from Paciente_Programas__c where Id = 'a1Y8c00000C5asnEAB' 
    """
)
derttyuiop = pd.DataFrame(results["records"])
derttyuiop.to_excel(
        "F:/Empresas SF/reportes/aaaaaa.xlsx", index=False
    )


In [74]:
# Recuperar empresas ID de salesforce desde Account
results = sf.query_all(
    """
  select Id, name, Holding__c ,Campa_a__c,Id_Empresa__c,Empleador__c,poliza__c,RUT_Empresa__c  from account where ispersonaccount = false 
    """
)
Empresas_SF = pd.DataFrame(results["records"])
Empresas_SF = Empresas_SF[
    [
        "Id",
        "Name",
        "Holding__c",
        "Campa_a__c",
        "Id_Empresa__c",
        "Empleador__c",
        "poliza__c",
        "RUT_Empresa__c",
    ]
]
Empresas_SF.shape

(7452, 8)

In [75]:
# buscar empresas desde Programas_por_Empresas__c datos de empresas
results = sf.query_all(
    """
   select Id,Empresa__c, Empresa_Padre__c,Nombre_Empresa__c, Name  from Programas_por_Empresas__c 
    """
)
Programas_por_empresa_SF = pd.DataFrame(results["records"])

Programas_por_empresa_SF = Programas_por_empresa_SF[
    ["Id", "Empresa__c", "Empresa_Padre__c", "Nombre_Empresa__c", "Name"]
]
Programas_por_empresa_SF.shape

(7151, 5)

In [76]:
#Union de Programas Empresa y empresas
Programas_empresas = pd.merge(
    left=Empresas_SF,
    right=Programas_por_empresa_SF,
    how="left",
    left_on="Id",
    right_on="Empresa__c",
)
Programas_empresas = Programas_empresas.rename(
    columns={
        "Id_x": "Id Account Empresa",
        "Name_x": "Nombre Empresa",
        "Holding__c": "Holding",
        "Campa_a__c": "Campaña",
        "Empleador__c": "Empleador",
        "poliza__c": "Poliza",
        "RUT_Empresa__c":"Rut Empresa",
        "Id_y": "Programas_por_Empresas__c",
        "Id_Empresa__c": "Codigo EMP",
        "Name_y": "Codigo A",
    }
)
Programas_empresas = Programas_empresas[
    [
        "Id Account Empresa",
        "Nombre Empresa",
        "Holding",
        "Campaña",
        "Empleador",
        "Poliza",
        "Rut Empresa",
        "Programas_por_Empresas__c",
        "Codigo EMP",
        "Codigo A"
    ]
]

Programas_empresas.shape

(7458, 10)

fin empresas y comienzo de rescate de pacientes


In [77]:
# CArga de PAcientes
results = sf.query_all(
    """
    select Id, IdentificationId__pc,Name, Identification_Type__c, Ocupaci_n__c,  PersonTitle ,PersonDepartment,HealthCloudGA__Gender__pc,Phone  from Account where ispersonaccount = True
   """
)
Pacientes_SF = pd.DataFrame(results["records"])

Pacientes_SF = Pacientes_SF[
    [
        "Id",
        "IdentificationId__pc",
        "Name",
        "Identification_Type__c",
        "Ocupaci_n__c",
        "PersonTitle",
        "PersonDepartment",
        "HealthCloudGA__Gender__pc",
        "Phone",
    ]
]
Pacientes_SF = Pacientes_SF.rename(
    columns={
        "Id": "Id Account Paciente",
        "IdentificationId__pc": "Rut",
        "Name": "Nombre",
        "Ocupaci_n__c": "Ocupacion",
        "PersonTitle": "Cargo",
        "PersonDepartment": "Departamento",
        "HealthCloudGA__Gender__pc": "Sexo",
        "Phone": "Telefono",
        "Identification_Type__c": "Tipo de Documento",
    }
)
Pacientes_SF.shape

(1787355, 9)

Aca se comienzan a unir las relaciones de pacientes y empresas ya creadas.

In [78]:
# buscar empresas desde Paciente_Programas__c relaciones de empresas con pacientes
results = sf.query_all(
    """
    select Id,Paciente__c, Programas_por_Empresas__c,padre__c from Paciente_Programas__c  
    """
)
Paciente_Programas_SF = pd.DataFrame(results["records"])

Paciente_Programas_SF = Paciente_Programas_SF[
    ["Id", "Paciente__c", "Programas_por_Empresas__c", "padre__c"]
]
Paciente_Programas_SF = Paciente_Programas_SF.rename(
    columns={
        "Id": "Id Pacientes_Programas__c",
        "Paciente__c": "Id Account",
        "Programas_por_Empresas__c": "Programas_por_Empresas__c",
        "padre__c": "Nombre Campaña",
    }
)
Paciente_Programas_SF.shape

(1390794, 4)

In [79]:
#Union de Pacientes Programas y Programas por empresa
empresas_unidas = pd.merge(
    left=Paciente_Programas_SF,
    right=Programas_empresas,
    how="left",
    left_on="Programas_por_Empresas__c",
    right_on="Programas_por_Empresas__c", 
)
empresas_unidas.shape

(1390794, 13)

In [81]:
Programas_empresas.to_excel("F:/Empresas SF/reportes/empresas.xlsx",index=False)

In [82]:
empresas_unidas.head()

,Id Pacientes_Programas__c,Id Account,Programas_por_Empresas__c,Nombre Campaña,Id Account Empresa,Nombre Empresa,Holding,Campaña,Empleador,Poliza,Rut Empresa,Codigo EMP,Codigo A
0,a1Y8c00000C5asnEAB,0018c00002amLbpAAE,a1W8c000007vDPvEAM,MetLife,0018c00002axQaeAAE,Serv. De Comerc. Y Ases. Callbusiness SPA,DBCH - CNSTD 2019,MetLife,Serv. De Comerc. Y Ases. Callbusiness SPA,340016490,None,EMP6008,A-0173
1,a1Y8c00000C5asoEAB,0018c00002amLbqAAE,a1W8c000007vDSpEAM,MetLife,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,None,EMP6274,A-0353
2,a1Y8c00000C5aspEAB,0018c00002amLbrAAE,a1W8c000007vDSpEAM,MetLife,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,None,EMP6274,A-0353
3,a1Y8c00000C5asqEAB,0018c00002amLbsAAE,a1W8c000007vDSpEAM,MetLife,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,None,EMP6274,A-0353
4,a1Y8c00000C5asrEAB,0018c00002amLc2AAE,a1W8c000007vDSpEAM,MetLife,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,None,EMP6274,A-0353


In [83]:
#pacientes_empresas22 = pd.merge(
    #left=Pacientes_SF,
    #right=empresas_unidas,
    #how="left",
    #left_on="Id Account Paciente",
    #right_on="Id Account", 
#)
#pacientes_empresas22 = pacientes_empresas22.drop_duplicates(['Rut'], keep='last')
#pacientes_empresas22 = pacientes_empresas22[
    #["Rut", "Nombre", "Nombre Campaña", "Nombre Empresa"]
#]


In [84]:
#pacientes_empresas22.shape

In [85]:
#apprix_1 = pacientes_empresas22.iloc[:848826,:]

#apprix_1.to_excel("F:/Empresas SF/reportes/Ruts_habilitados1.xlsx",index=False)


In [86]:
#apprix_2 = pacientes_empresas22.iloc[848827:,:]#
#apprix_2.to_excel("F:/Empresas SF/reportes/Ruts_habilitados2.xlsx",index=False)

In [87]:
#pacientes_empresas22.head()

Carga de Archivo carga

In [88]:
nuevos = "F:/Empresas SF/reportes/carga.xlsx"
campaña = "THB Seguros"
Pais = "Chile"
#Pais = "México"

Pacientes_Archivo = pd.read_excel(nuevos)
# OJO se copia rut1 a rut le saca el guoion


In [89]:
Pacientes_Archivo.head(10)

,CONTRATANTE,poliza,GRUPO,RUT,DV,RUT CARGA,DV CARGA,rut1,APELLIDO PATERNO,APELLIDO MATERNO,apellidos,nombres,nacimientoxx,nacimiento,RELACION,FECHA INICIO VIGENCIA
0,Psc Ingenieria S.A.,76585040-1,Asegurados,14435582,2,14435582,2,14435582-2,Aguila,Morales,Aguila Morales,Osvaldo Guillermo,15-02-1976,1976-02-15,Asegurado,17-04-2015
1,Psc Ingenieria S.A.,76585040-1,Asegurados,14435582,2,13458005,4,13458005-4,Naranjo,Vasquez,Naranjo Vasquez,Maria Francisca,31-08-1978,1978-08-31,Conyuge,17-04-2015
2,Psc Ingenieria S.A.,76585040-1,Asegurados,14435582,2,23395138,2,23395138-2,Aguila,Naranjo,Aguila Naranjo,Magdalena Amparo,02-08-2010,2010-08-02,Hijo,17-04-2015
3,Psc Ingenieria S.A.,76585040-1,Asegurados,14435582,2,25866959,2,25866959-2,Aguila,Naranjo,Aguila Naranjo,Agustin Andres,04-08-2017,2017-08-04,Hijo,04-08-2017
4,Psc Ingenieria S.A.,76585040-1,Asegurados,12218164,2,12218164,2,12218164-2,Andrade,Said,Andrade Said,Luis Alberto,14-06-1972,1972-06-14,Asegurado,01-11-2018
5,Psc Ingenieria S.A.,76585040-1,Asegurados,12218164,2,13906259,0,13906259-0,Retamales,Gallegos,Retamales Gallegos,Ximena,14-10-1980,1980-10-14,Conyuge,01-11-2018
6,Psc Ingenieria S.A.,76585040-1,Asegurados,12218164,2,24245697,1,24245697-1,Andrade,Retamales,Andrade Retamales,Julieta,12-04-2013,2013-04-12,Hijo,01-11-2018
7,Psc Ingenieria S.A.,76585040-1,Asegurados,12218164,2,26147724,6,26147724-6,Andrade,Retamales,Andrade Retamales,Josefina,01-03-2018,2018-03-01,Hijo,01-11-2018
8,Psc Ingenieria S.A.,76585040-1,Asegurados,12455156,0,12455156,0,12455156-0,Arenas,Delano,Arenas Delano,Rodrigo,19-07-1973,1973-07-19,Asegurado,01-10-2019
9,Psc Ingenieria S.A.,76585040-1,Asegurados,12455156,0,13275751,8,13275751-8,Araya,Gonzalez,Araya Gonzalez,Lissette,16-01-1977,1977-01-16,Conyuge,01-06-2022


In [90]:
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut1"].astype(str)


In [91]:
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.replace("-", "")

Pacientes_Archivo.head()

,CONTRATANTE,poliza,GRUPO,RUT,DV,RUT CARGA,DV CARGA,rut1,APELLIDO PATERNO,APELLIDO MATERNO,apellidos,nombres,nacimientoxx,nacimiento,RELACION,FECHA INICIO VIGENCIA,rut
0,Psc Ingenieria S.A.,76585040-1,Asegurados,14435582,2,14435582,2,14435582-2,Aguila,Morales,Aguila Morales,Osvaldo Guillermo,15-02-1976,1976-02-15,Asegurado,17-04-2015,144355822
1,Psc Ingenieria S.A.,76585040-1,Asegurados,14435582,2,13458005,4,13458005-4,Naranjo,Vasquez,Naranjo Vasquez,Maria Francisca,31-08-1978,1978-08-31,Conyuge,17-04-2015,134580054
2,Psc Ingenieria S.A.,76585040-1,Asegurados,14435582,2,23395138,2,23395138-2,Aguila,Naranjo,Aguila Naranjo,Magdalena Amparo,02-08-2010,2010-08-02,Hijo,17-04-2015,233951382
3,Psc Ingenieria S.A.,76585040-1,Asegurados,14435582,2,25866959,2,25866959-2,Aguila,Naranjo,Aguila Naranjo,Agustin Andres,04-08-2017,2017-08-04,Hijo,04-08-2017,258669592
4,Psc Ingenieria S.A.,76585040-1,Asegurados,12218164,2,12218164,2,12218164-2,Andrade,Said,Andrade Said,Luis Alberto,14-06-1972,1972-06-14,Asegurado,01-11-2018,122181642


In [92]:
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].astype(str)
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.replace(".", "")

# aca se trabaja con el nuevo rut desde rut1 ojooooooo
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.upper()

Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.strip()
Pacientes_Archivo.head()

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_16468\1949753364.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.replace(".", "")


,CONTRATANTE,poliza,GRUPO,RUT,DV,RUT CARGA,DV CARGA,rut1,APELLIDO PATERNO,APELLIDO MATERNO,apellidos,nombres,nacimientoxx,nacimiento,RELACION,FECHA INICIO VIGENCIA,rut
0,Psc Ingenieria S.A.,76585040-1,Asegurados,14435582,2,14435582,2,14435582-2,Aguila,Morales,Aguila Morales,Osvaldo Guillermo,15-02-1976,1976-02-15,Asegurado,17-04-2015,144355822
1,Psc Ingenieria S.A.,76585040-1,Asegurados,14435582,2,13458005,4,13458005-4,Naranjo,Vasquez,Naranjo Vasquez,Maria Francisca,31-08-1978,1978-08-31,Conyuge,17-04-2015,134580054
2,Psc Ingenieria S.A.,76585040-1,Asegurados,14435582,2,23395138,2,23395138-2,Aguila,Naranjo,Aguila Naranjo,Magdalena Amparo,02-08-2010,2010-08-02,Hijo,17-04-2015,233951382
3,Psc Ingenieria S.A.,76585040-1,Asegurados,14435582,2,25866959,2,25866959-2,Aguila,Naranjo,Aguila Naranjo,Agustin Andres,04-08-2017,2017-08-04,Hijo,04-08-2017,258669592
4,Psc Ingenieria S.A.,76585040-1,Asegurados,12218164,2,12218164,2,12218164-2,Andrade,Said,Andrade Said,Luis Alberto,14-06-1972,1972-06-14,Asegurado,01-11-2018,122181642


In [93]:
if ("nombres" in Pacientes_Archivo.columns) == True:
    Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace(",", "")
    Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace(".", "")
    Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace("-", "")
    Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.title()
    Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.strip()
# Pacientes_Archivo["empresa"] = Pacientes_Archivo["empresa"].str.title()
    Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace(",", "")
    Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace(".", "")
    Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace("-", "")
    Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.title()
    Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.strip()

Pacientes_SF["Rut"] = Pacientes_SF["Rut"].astype(str)
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.strip()
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.upper()
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.replace("-", "")


Pacientes_Archivo["RutEstaoNo"] = Pacientes_Archivo["rut"].isin(Pacientes_SF["Rut"])

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_16468\192164855.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace(".", "")
C:\Users\jmartinez\AppData\Local\Temp\ipykernel_16468\192164855.py:9: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace(".", "")


In [94]:
#aca a los pacientes del archivo se les asigna sus datos en SF
Pacientes_con_id = pd.merge(
    left=Pacientes_Archivo,
    right=Pacientes_SF,
    how="left",
    left_on="rut",
    right_on="Rut",
)
Pacientes_con_id.shape

(138, 27)

In [95]:
Pacientes_con_id.head()

,CONTRATANTE,poliza,GRUPO,RUT,DV,RUT CARGA,DV CARGA,rut1,APELLIDO PATERNO,APELLIDO MATERNO,...,RutEstaoNo,Id Account Paciente,Rut,Nombre,Tipo de Documento,Ocupacion,Cargo,Departamento,Sexo,Telefono
0,Psc Ingenieria S.A.,76585040-1,Asegurados,14435582,2,14435582,2,14435582-2,Aguila,Morales,...,True,0018c00002eVzgWAAS,144355822,Osvaldo Guillermo Aguila Morales,RUT,None,None,None,Hombre,None
1,Psc Ingenieria S.A.,76585040-1,Asegurados,14435582,2,13458005,4,13458005-4,Naranjo,Vasquez,...,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Psc Ingenieria S.A.,76585040-1,Asegurados,14435582,2,23395138,2,23395138-2,Aguila,Naranjo,...,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Psc Ingenieria S.A.,76585040-1,Asegurados,14435582,2,25866959,2,25866959-2,Aguila,Naranjo,...,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Psc Ingenieria S.A.,76585040-1,Asegurados,12218164,2,12218164,2,12218164-2,Andrade,Said,...,True,0018c00002ehM7xAAE,122181642,Andrade Said Luis Alberto,RUT,None,None,None,Otros,None


In [96]:
# Final_pacientes_y_empresa = Pacientes_con_ID
Subir_pacientes = pd.DataFrame()
Subir_pacientes["RutEstaoNo"] = Pacientes_con_id["RutEstaoNo"]
# agrupar = empresasqfaltan[empresasqfaltan['si']==False]


if ("nombres" in Pacientes_con_id.columns) == True:
    Subir_pacientes["FirstName"] = Pacientes_con_id["nombres"]
if ("apellidos" in Pacientes_con_id.columns) == True:
    Subir_pacientes["LastName"] = Pacientes_con_id["apellidos"]
if ("rut" in Pacientes_con_id.columns) == True:
    Subir_pacientes["IdentificationId__pc"] = (
        Pacientes_con_id["rut"].astype(str).str.upper()
    )
    Subir_pacientes.head(20)
if ("email" in Pacientes_con_id.columns) == True:
    # Pacientes_con_id["email"] = Pacientes_con_id["email"].astype(str).str.lower().replace("nan","")
    Subir_pacientes["PersonEmail"] = (
        Pacientes_con_id["email"].astype(str).str.lower().replace("nan", "")
    )
if ("nacimiento" in Pacientes_con_id.columns) == True:
    Subir_pacientes["PersonBirthdate"] = Pacientes_con_id["nacimiento"]

if ("sexo" in Pacientes_con_id.columns) == True:
    Sexo = {
        "M": "Hombre",
        "m": "Hombre",
        "Masculino": "Hombre",
        "Femenino": "Mujer",
        "": "Prefiero no responder",
        "HOMBRE": "Hombre",
        "MUJER": "Mujer",
        "F": "Mujer",
        "O": "O",
        "Ambos": "Prefiero no responder",
        "N": "Prefiero no responder",
        "Otro": "Prefiero no responder",
        "n": "Prefiero no responder",
        "N": "Prefiero no responder",
        "FEMENINO": "Mujer",
        "f": "Mujer",
        "MASCULINO": "Hombre",
        "m": "Hombre",
        "M ": "Hombre",
        "F ": "Mujer",
        "Varón": "Hombre",
        "varon":"Hombre"
    }
    Pacientes_con_id.replace({"sexo": Sexo}, inplace=True)
    Subir_pacientes["HealthCloudGA__Gender__pc"] = Pacientes_con_id["sexo"]
if ("direccion" in Pacientes_con_id.columns) == True:
    Subir_pacientes["BillingStreet"] = (
        Pacientes_con_id["direccion"].str.title().replace("NaN", "")
    )
    # Subir_pacientes["BillingStreet"] = Final_pacientes_y_empresa["direccion"]
if ("comuna" in Pacientes_con_id.columns) == True:
    # Subir_pacientes["BillingState"] = Final_pacientes_y_empresa["comuna"]
    Subir_pacientes["BillingState"] = (
        Pacientes_con_id["comuna"].str.title().replace("NaN", "")
    )
    # Subir_pacientes["BillingState"] = Final_pacientes_y_empresa["comuna"]
if ("ciudad" in Pacientes_con_id.columns) == True:
    # Subir_pacientes["BillingCity"] = Final_pacientes_y_empresa["ciudad"]
    Subir_pacientes["BillingCity"] = (
        Pacientes_con_id["ciudad"].str.title().replace("NaN", "")
    )

if ("cargo" in Pacientes_con_id.columns) == True:
    Subir_pacientes["PersonTitle"] = Pacientes_con_id["cargo"].str.title()

if ("division" in Pacientes_con_id.columns) == True:
    Subir_pacientes["PersonDepartment"] = Pacientes_con_id["division"].str.title()

if ("celular" in Pacientes_con_id.columns) == True:
    Subir_pacientes["Phone"] = (
        Pacientes_con_id["celular"].apply(str).str.strip().replace("+", "")
    )
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("NaN", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("Nan", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("-", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("0", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("No Informado", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("-", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("+", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("nan", "")

Subir_pacientes["RecordTypeId"] = "0128c000002Byp4AAC"
Subir_pacientes["Cuenta_validada__c"] = "True"


if Pais == "Chile":
    Subir_pacientes["BillingCountry"] = "Chile"
    Subir_pacientes["Identification_Type__c"] = "RUT"
    Subir_pacientes["Pa_s_del_Paciente__c"] = "a1U8c000005dAtnEAE"
else:
    Subir_pacientes["BillingCountry"] = "México"
    Subir_pacientes["Identification_Type__c"] = "CURP"
    Subir_pacientes["Pa_s_del_Paciente__c"] = "a1U8c000005dAtsEAE"


Subir_pacientes = Subir_pacientes[Subir_pacientes["RutEstaoNo"] == False]


# Subir_pacientes.PersonEmail = Subir_pacientes.PersonEmail.fillna('')
Subir_pacientes.to_excel("F:/Empresas SF/reportes/account.xlsx", index=False)
Subir_pacientes.head()

,RutEstaoNo,FirstName,LastName,IdentificationId__pc,PersonBirthdate,RecordTypeId,Cuenta_validada__c,BillingCountry,Identification_Type__c,Pa_s_del_Paciente__c
1,False,Maria Francisca,Naranjo Vasquez,134580054,1978-08-31,0128c000002Byp4AAC,True,Chile,RUT,a1U8c000005dAtnEAE
2,False,Magdalena Amparo,Aguila Naranjo,233951382,2010-08-02,0128c000002Byp4AAC,True,Chile,RUT,a1U8c000005dAtnEAE
3,False,Agustin Andres,Aguila Naranjo,258669592,2017-08-04,0128c000002Byp4AAC,True,Chile,RUT,a1U8c000005dAtnEAE
5,False,Ximena,Retamales Gallegos,139062590,1980-10-14,0128c000002Byp4AAC,True,Chile,RUT,a1U8c000005dAtnEAE
6,False,Julieta,Andrade Retamales,242456971,2013-04-12,0128c000002Byp4AAC,True,Chile,RUT,a1U8c000005dAtnEAE


In [97]:
clinica_virtual = pd.DataFrame()
clinica_virtual["rut"] = Pacientes_con_id["rut1"].str.upper().str.strip()
clinica_virtual["firstName"] = Pacientes_con_id["nombres"].str.title().str.strip()
clinica_virtual["lastName"] = Pacientes_con_id["apellidos"].str.title().str.strip()
clinica_virtual["accountId"] = campaña
fecha_arc = datetime.now()
dia_hoy = str(fecha_arc.day)
mes_hoy = str(fecha_arc.month)



Codigos_Clinica_Virtual = {
    "BCI": "1",
    "Farmacias Ahumada": "2",
    "MetLife": "3",
    "Vida Security": "4",
    "Willis": "5",
    "Entel": "6",
    "Testing": "7",
    "3SP": "8",
    "HomeCare": "9",
    "Pedro de Valdivia": "10",
    "Colaboradores": "11",
    "MineraAbra": "12",
    "CMPC": "13",
    "Lipigas": "14",
    "Asismed": "15",
    "Siempre Perú": "16",
    "Farmacias Cruz Verde": "17",
    "WFS Food Services": "18",
    "VidaCel": "19",
    "Dermik": "20",
    "EY": "21",
    "IND": "22",
    "Sindicato Walmart": "23",
    "Instruvalve": "24",
    "VidaCel Colaboradores": "25",
    "Socofar": "26",
    "Milan Fajanovic": "27",
    "THBseguros": "28",
    "Pesco": "29",
    "Burgos - Consorcio": "30",
    "COFAMA-SILENTIUM": "31",
    "KAUFMANN": "32",
    "QuePlan": "33",
    "Zurich": "34",
    "Colaboradores Siempre Chile": "35",
    "Talo": "36",
    "Transvip": "37",
    "Vida Security Colaboradores": "38",
    "FORUS": "39",
    "Sura": "40",
    "Southbrige": "41",
    "Construmart": "42",
    "BICE Vida": "43",
    "Lockton": "44",
    "Mercer": "48",
    "Grupo PSG": "46",
    "Siempre Chile": "47",
}

clinica_virtual.replace({"accountId": Codigos_Clinica_Virtual}, inplace=True)
#clinica_virtual.to_excel("F:/Empresas SF/reportes/Clinica_Virtual.xlsx", index=False)
clinica_virtual.to_excel("F:/Cargas Base Datos CRM/Altas CV/Sin Cargar/Clinica_Virtual_"+campaña+"_"+dia_hoy+"_"+mes_hoy+".xlsx", index=False)


In [98]:
print(dia_hoy)

20


In [99]:
campaña_seleccionada = empresas_unidas[empresas_unidas["Campaña"] == campaña]
campaña_seleccionada.head()


,Id Pacientes_Programas__c,Id Account,Programas_por_Empresas__c,Nombre Campaña,Id Account Empresa,Nombre Empresa,Holding,Campaña,Empleador,Poliza,Rut Empresa,Codigo EMP,Codigo A
158865,a1Y8c00000C6VjSEAV,0018c00002amVQXAA2,a1W8c0000083iWsEAI,THB Seguros,0018c00002cXJU2AAO,Addval,Sin Holding Asociado,THB Seguros,Addval,Sin Póliza Asociada,None,EMP7483,A-0732
158866,a1Y8c00000C6VjTEAV,0018c00002amVQcAAM,a1W8c0000083iWsEAI,THB Seguros,0018c00002cXJU2AAO,Addval,Sin Holding Asociado,THB Seguros,Addval,Sin Póliza Asociada,None,EMP7483,A-0732
158867,a1Y8c00000C6VjVEAV,0018c00002amVkFAAU,a1W8c0000083iWsEAI,THB Seguros,0018c00002cXJU2AAO,Addval,Sin Holding Asociado,THB Seguros,Addval,Sin Póliza Asociada,None,EMP7483,A-0732
158868,a1Y8c00000C6VjXEAV,0018c00002amQHBAA2,a1W8c0000083iWsEAI,THB Seguros,0018c00002cXJU2AAO,Addval,Sin Holding Asociado,THB Seguros,Addval,Sin Póliza Asociada,None,EMP7483,A-0732
158869,a1Y8c00000C6VjaEAF,0018c00002amR2xAAE,a1W8c0000083idGEAQ,THB Seguros,0018c00002cXJaVAAW,Desert King Chile,Sin Holding Asociado,THB Seguros,Desert King Chile,Sin Póliza Asociada,None,EMP6850,A-1128


In [100]:
campaña_seleccionada.shape

(6382, 13)

In [101]:
if campaña != "MetLife":
    pacientes_en_campaña = pd.merge(
        left=campaña_seleccionada,
        right=Pacientes_con_id,
        how="left",
        left_on="Id Account",
        right_on="Id Account Paciente",
    )
    pacientes_en_campaña["Paciente__c"] = "Id Pacientes"
    pacientes_en_campaña["Programas_por_Empresas__c"] = "Id Programas Por empresa"
    pacientes_en_campaña.to_excel(
        "F:/Empresas SF/reportes/campaña_empresa.xlsx", index=False
    )
    campaña_en_pacientes = pd.merge(
        left=Pacientes_con_id,
        right=campaña_seleccionada,
        how="left",
        left_on="Id Account Paciente",
        right_on="Id Account",
    )
    campaña_en_pacientes.to_excel(
        "F:/Empresas SF/reportes/archivo_campaña.xlsx", index=False
    )

In [102]:
if campaña != "MetLife1":
    campaña_en_pacientes = pd.merge(
        left=Pacientes_con_id,
        right=campaña_seleccionada,
        how="left",
        left_on="Id Account Paciente",
        right_on="Id Account",
    )
    if ("poliza" in campaña_en_pacientes.columns) == True:
        campaña_en_pacientes["poliza"] = campaña_en_pacientes["poliza"].astype(str)

    campaña_en_pacientes = campaña_en_pacientes.rename(
        columns={"Programas_por_Empresas__c": "programas por empresa"}
    )
    campaña_en_pacientes["Paciente__c"] = campaña_en_pacientes["Id Account Paciente"]
    campaña_en_pacientes["Programas_por_Empresas__c"] = campaña_en_pacientes[
        "programas por empresa"
    ]
    campaña_en_pacientes.to_excel(
        "F:/Empresas SF/reportes/archivo_campaña2.xlsx", index=False
    )

In [103]:

#apprix_1 = Pacientes_SF.iloc[:790552,:]
#apprix_2 = Pacientes_SF.iloc[790553:,:]
#apprix_1.to_excel("F:/Empresas SF/reportes/Ruts1.xlsx",index=False)
#apprix_2.to_excel("F:/Empresas SF/reportes/Ruts2.xlsx",index=False)


In [104]:
campaña_en_pacientes2 = pd.merge(
    left=Pacientes_con_id,
    right=campaña_seleccionada,
    how="left",
    left_on="Id Account Paciente",
    right_on="Id Account",
)
if ("poliza" in campaña_en_pacientes2.columns) == True:
    campaña_en_pacientes2["poliza"] = campaña_en_pacientes2["poliza"].astype(str)

campaña_en_pacientes2 = campaña_en_pacientes2.rename(
    columns={"Programas_por_Empresas__c": "programas por empresa"}
)
campaña_en_pacientes2["Paciente__c"] = campaña_en_pacientes2["Id Account Paciente"]
campaña_en_pacientes2["Programas_por_Empresas__c"] = campaña_en_pacientes2[
    "programas por empresa"
]


""" campaña_en_pacientes2 = campaña_en_pacientes2[
    (campaña_en_pacientes2["Programas_por_Empresas__c"] == "")] """
campaña_en_pacientes2["Esta"] = campaña_en_pacientes2["programas por empresa"].isnull()
campaña_en_pacientes2 = campaña_en_pacientes2[campaña_en_pacientes2["Esta"]]
campaña_en_pacientes2.to_excel(
    "F:/Empresas SF/reportes/archivo_campaña3.xlsx", index=False
)